In [ ]:
from mlx_lm import load, generate

def add_template(prompt):
    return f'''
    <|start_header_id|>user<|end_header_id|>

    {prompt}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    '''

response = generate(model, tokenizer, prompt=add_template("What is total return swap"), verbose=True, max_tokens=500)

/Users/jianlu/anaconda3/envs/ML_general/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jianlu/anaconda3/envs/ML_general/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 122164.19it/s]


Prompt: 
    <|start_header_id|>user<|end_header_id|>

    What is total return swap<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    
A Total Return Swap (TRS) is a type of financial derivative that allows two parties to exchange the total return of two different assets, such as a bond and a stock, over a specified period of time. The TRS is used to manage risk, generate income, and enhance returns in a portfolio.

Here's how it works:

1. **Two parties**: A TRS involves two parties: the buyer and the seller. The buyer is typically an investor or a fund manager who wants to gain exposure to a particular asset or market, while the seller is usually a financial institution or a counterparty that provides the swap.
2. **Reference assets**: The buyer and seller agree on two reference assets, such as a bond and a stock. The total return of these assets is used as the basis for the swap.
3. **Swap agreement**: The buyer and seller agree on the terms of the swap, including the

In [1]:
from lora import load, generate, lora_utils

def add_template(prompt):
    return f'''
    <|start_header_id|>user<|end_header_id|>

    {prompt}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    '''
model_name = '/Volumes/PS2000/huggingface/hub/models--mlx-community--Meta-Llama-3-8B-Instruct-4bit/snapshots/c38b3b1f03cce0ce0ccd235e5c97b0d3d255e651'
adapter_file = '/Users/jianlu/Documents/GitHub/mlx-examples/lora/LoRA_GMS8.npz'
tokenizer_config = {}
model, tokenizer, _ = lora_utils.load(model_name, tokenizer_config)
print(f"Loading Lora Adapter from {adapter_file}")
model.load_weights(adapter_file, strict=False)


/Users/jianlu/anaconda3/envs/ML_general/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jianlu/anaconda3/envs/ML_general/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading Lora Adapter from /Users/jianlu/Documents/GitHub/mlx-examples/lora/LoRA_GMS8.npz


Model(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): QuantizedLinear(input_dims=4096, output_dims=4096, bias=False,group_size=64, bits=4)
        (k_proj): QuantizedLinear(input_dims=4096, output_dims=1024, bias=False,group_size=64, bits=4)
        (v_proj): QuantizedLinear(input_dims=4096, output_dims=1024, bias=False,group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=4096, output_dims=4096, bias=False,group_size=64, bits=4)
        (rope): RoPE(128, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): QuantizedLinear(input_dims=4096, output_dims=14336, bias=False,group_size=64, bits=4)
        (down_proj): QuantizedLinear(input_dims=14336, output_dims=4096, bias=False,group_size=64, bits=4)
        (up_proj): QuantizedLinear(input_dims=4096, output_dims=14336, bias=False,group_size=64, bits=4)
      )
      (input_layernorm): RMSNorm(4096, eps=1e-05)
  

In [ ]:
from datasets import load_dataset
import ollama
import json
# ds = load_dataset("qintongli/GSM-Plus")
ds = load_dataset("openai/gsm8k", "main")
i = 0
f = open('GSM8k_LoRA_results.jsonl', 'w')
for q in ds['test']:
    question = q['question']
    label = q['answer']
    
    length = len(add_template(q['question']))
    while True:
        generated_answer = generate(model, add_template(q['question']), tokenizer) # Lora
        if len(generated_answer) > 0:
            break

    print(f"question {i} finished, total {len(ds['test'])}")
    i += 1
    prompt = f''' 
    You are given a predicted answer and a ground truth solution, determinant whether the predicted answer match the ground truth answer. End your response with <True> or <False>
    predicted answer: {generated_answer}
    ground truth solution:
    {label}
    '''
    response = ollama.generate(model='llama3.1', prompt=prompt)
    evaluation = response['response']

    new_data = {'question': q['question'], 'solution': q['answer'], 'generated_answer': generated_answer, 'evaluation': evaluation}
    f.write(json.dumps(new_data) + '\n')




    <|start_header_id|>user<|end_header_id|>

    Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    
No tokens generated for this prompt

    <|start_header_id|>user<|end_header_id|>

    Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    
No tokens generated for this prompt

    <|start_header_id|>user<|end_header_id|>

    Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and b